# Ectraction, Transformation and Loading of Vessel Data

links
- https://commons.wikimedia.org/wiki/Category:IMO_1000150
- https://glamtools.toolforge.org/glamorgan.html?&category=IMO_9802683&depth=1&month=last
- https://commons.wikimedia.org/w/index.php?title=Category:Ships_by_IMO_number&subcatfrom=5029738%0AIMO+5029738#mw-subcategories

# Contents

1. Importing dependecies
2. Check the URL
3. Main Loop
4. Save the Data to database

In [1]:
from bs4 import BeautifulSoup as bs
from html_table_parser.parser import HTMLTableParser
import urllib.request
import pandas as pd
import re
import requests
import os
import locale
from datetime import datetime
locale.setlocale(locale.LC_ALL, 'en_US') # To deal with months later
from google.cloud import bigquery
from google.oauth2 import service_account 
import html5lib

D:\anaconda\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
C:\Users\DELL\AppData\Local\Temp\ipykernel_4936\961921699.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Webscraping IMO Numbers. 

I am going to scrape the IMO Numbers from wikipedia. 

In [2]:
page_number=[1000021,1010557,1010557,5002508,5029738,5056171,5090593,5117298,5145972,5170551,5201752,5228530,5261154,5285136,
             5315462,5345596,5377185,5408099,5604840,6414966,6515332,6611681,6711716,6806444,6904117,7005009,7032246,7111913,
             7213137,7302885,7334199,7361233,7389924,7414016,7504158,7531979,7626592,7706419,7725611,7805136,7824871,7915266,
             7941746,8016811,8105002,8126575,8208660,8225022,8313661,8333568,8412699,8433435,8511550,8602012,8616568,8640985,
             8700632,8713641,8731734,8762587,8806682,8821797,8845999,8878257,8906614,8917481,8937053,8966315,8987694,9005479,
             9015931,9034705,9048122,9061590,9076753,9090292,9107679,9116357,9125085,9133824,9141340,9148453,9156395,9164299,
             9172648,9179933,9187394,9193288,9199165,9207675,9214991,9222041,9227857,9232967,9238753,9245469,9251561,9257838,
             9264726,9272931,9279604,9285201,9291523,9297797,9303340,9309227,9315018,9321017,9327138,9333474,9339595,9346641,
             9353125,9360051,9366263,9373888,9381158,9388637,9396567,9404297,9412517,9420863,9428451,9435478,9435674,9443164,
             9450791,9460136,9467770,9477828,9487055,9498274,9508067,9520285,9531856,9543316,9555905,9571105,9583768,9594248,
             9604809,9614983,9624938,9636436,9648697,9660114,9671589,9685229,9696553,9706906,9719068,9731602,9744764,9758571,
             9772565, 9786152,9802619,9818278,9837456,9856490,9885893,9885893
]

In [3]:
for i in range(len(page_number)):
    url = f"https://commons.wikimedia.org/w/index.php?title=Category:Ships_by_IMO_number&subcatuntil={page_number[i]}%0AIMO+{page_number[i]}#mw-subcategories"
    print(url)
    i = i + 1

https://commons.wikimedia.org/w/index.php?title=Category:Ships_by_IMO_number&subcatuntil=1000021%0AIMO+1000021#mw-subcategories
https://commons.wikimedia.org/w/index.php?title=Category:Ships_by_IMO_number&subcatuntil=1010557%0AIMO+1010557#mw-subcategories
https://commons.wikimedia.org/w/index.php?title=Category:Ships_by_IMO_number&subcatuntil=1010557%0AIMO+1010557#mw-subcategories
https://commons.wikimedia.org/w/index.php?title=Category:Ships_by_IMO_number&subcatuntil=5002508%0AIMO+5002508#mw-subcategories
https://commons.wikimedia.org/w/index.php?title=Category:Ships_by_IMO_number&subcatuntil=5029738%0AIMO+5029738#mw-subcategories
https://commons.wikimedia.org/w/index.php?title=Category:Ships_by_IMO_number&subcatuntil=5056171%0AIMO+5056171#mw-subcategories
https://commons.wikimedia.org/w/index.php?title=Category:Ships_by_IMO_number&subcatuntil=5090593%0AIMO+5090593#mw-subcategories
https://commons.wikimedia.org/w/index.php?title=Category:Ships_by_IMO_number&subcatuntil=5117298%0AIMO+5

In [4]:
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}

imo_numbers=[]
for i in range(len(page_number)):
    url=f"https://commons.wikimedia.org/w/index.php?title=Category:Ships_by_IMO_number&subcatuntil={page_number[i]}%0AIMO+{page_number[i]}#mw-subcategories"
    r = requests.get(url, header)
    html=r.content
    soup=bs(html,"lxml")
    
    for i in soup.find_all('div', {'class':'CategoryTreeItem'}):
        a_tag_text=i.find('a').get_text(strip=True)
        imo_numbers.append(a_tag_text)
imo_numbers

['IMO 1000021',
 'IMO 1000150',
 'IMO 1000318',
 'IMO 1000540',
 'IMO 1000655',
 'IMO 1000863',
 'IMO 1000899',
 'IMO 1001130',
 'IMO 1001178',
 'IMO 1001180',
 'IMO 1001192',
 'IMO 1001269',
 'IMO 1001489',
 'IMO 1001506',
 'IMO 1001544',
 'IMO 1001702',
 'IMO 1001881',
 'IMO 1001934',
 'IMO 1001984',
 'IMO 1002017',
 'IMO 1002055',
 'IMO 1002213',
 'IMO 1002225',
 'IMO 1002275',
 'IMO 1002304',
 'IMO 1002378',
 'IMO 1002380',
 'IMO 1002445',
 'IMO 1002469',
 'IMO 1002495',
 'IMO 1002550',
 'IMO 1002603',
 'IMO 1002627',
 'IMO 1002720',
 'IMO 1002756',
 'IMO 1002782',
 'IMO 1002902',
 'IMO 1002990',
 'IMO 1003267',
 'IMO 1003308',
 'IMO 1003530',
 'IMO 1003827',
 'IMO 1003968',
 'IMO 1004106',
 'IMO 1004247',
 'IMO 1004493',
 'IMO 1004625',
 'IMO 1004675',
 'IMO 1004704',
 'IMO 1004819',
 'IMO 1004833',
 'IMO 1004871',
 'IMO 1004900',
 'IMO 1004936',
 'IMO 1005148',
 'IMO 1005150',
 'IMO 1005174',
 'IMO 1005253',
 'IMO 1005435',
 'IMO 1005576',
 'IMO 1005679',
 'IMO 1005734',
 'IMO 10

In [5]:
imo_list=[]
for i in range(len(imo_numbers)):
    imo_list.append(imo_numbers[i].replace('IMO ', ''))
imo_list

['1000021',
 '1000150',
 '1000318',
 '1000540',
 '1000655',
 '1000863',
 '1000899',
 '1001130',
 '1001178',
 '1001180',
 '1001192',
 '1001269',
 '1001489',
 '1001506',
 '1001544',
 '1001702',
 '1001881',
 '1001934',
 '1001984',
 '1002017',
 '1002055',
 '1002213',
 '1002225',
 '1002275',
 '1002304',
 '1002378',
 '1002380',
 '1002445',
 '1002469',
 '1002495',
 '1002550',
 '1002603',
 '1002627',
 '1002720',
 '1002756',
 '1002782',
 '1002902',
 '1002990',
 '1003267',
 '1003308',
 '1003530',
 '1003827',
 '1003968',
 '1004106',
 '1004247',
 '1004493',
 '1004625',
 '1004675',
 '1004704',
 '1004819',
 '1004833',
 '1004871',
 '1004900',
 '1004936',
 '1005148',
 '1005150',
 '1005174',
 '1005253',
 '1005435',
 '1005576',
 '1005679',
 '1005734',
 '1005746',
 '1005813',
 '1005899',
 '1005904',
 '1006013',
 '1006099',
 '1006128',
 '1006207',
 '1006245',
 '1006257',
 '1006295',
 '1006300',
 '1006324',
 '1006336',
 '1006374',
 '1006544',
 '1006556',
 '1006609',
 '1006673',
 '1006702',
 '1006805',
 '10

In [6]:
print(len(imo_list))

32200


In [12]:
import time
table_values=[]
for i in range(1000):
    try:
        url_2=f"https://commons.wikimedia.org/wiki/Category:IMO_{imo_list[i]}"
        res=requests.get(url_2, header)
        html=res.content
        soup=bs(html,"lxml")
        table=soup.find('table', id='wdinfobox')
        
        vessel_name=soup.find('caption', class_='fn org').find('b').get_text(strip=True)
        rows = table.find_all('tr')  # Replace with the actual row class
        for row in rows:
            cells = row.find_all('td')  # Assuming data is in 'td' elements
            data = [cell.text.strip() for cell in cells]
                
        table_values.append({ 'vessel_name':vessel_name, 'table_data': data})
    except:
        time.sleep(0.1)
        continue
        

In [13]:
table_values

[{'vessel_name': 'Montkaj', 'table_data': ['']},
 {'vessel_name': 'Adix', 'table_data': ['']},
 {'vessel_name': 'Al Riyadh', 'table_data': ['']},
 {'vessel_name': 'Asgard II', 'table_data': ['']},
 {'vessel_name': 'Amorina', 'table_data': ['']},
 {'vessel_name': 'Montecristo', 'table_data': ['']},
 {'vessel_name': 'Be Mine', 'table_data': ['']},
 {'vessel_name': 'Capella C', 'table_data': ['']},
 {'vessel_name': 'Minderella', 'table_data': ['']},
 {'vessel_name': 'Cedar Sea II', 'table_data': ['']},
 {'vessel_name': 'Bad Girl', 'table_data': ['']},
 {'vessel_name': 'Concordia', 'table_data': ['']},
 {'vessel_name': 'Rena', 'table_data': ['']},
 {'vessel_name': 'Enigma', 'table_data': ['']},
 {'vessel_name': 'Shahnaz', 'table_data': ['']},
 {'vessel_name': 'Carita', 'table_data': ['']},
 {'vessel_name': 'Golden Odyssey', 'table_data': ['']},
 {'vessel_name': 'Hasabi II', 'table_data': ['']},
 {'vessel_name': 'Il Vagabondo', 'table_data': ['']},
 {'vessel_name': 'My Legend', 'table_data'

In [18]:
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}

position = []
import random
imo_small_list=random.sample(imo_list, 10)

for IMO in imo_small_list:
    url = r'https://locator-tool.toolforge.org/#/geolocate?category=IMO_' + str(IMO)
    req = requests.get(url,header)
    html=req.content
    soup=bs(html, 'lxml')
    input_tag=soup.find('input', class_='form-control ng-pristine ng-valid ng-not-empty ng-touched')
    input_value=input_tag['value']
    position.append({ 'ship_imo': IMO,'lat_long': input_value })

print(position)
    

TypeError: 'NoneType' object is not subscriptable

In [ ]:
#get it on a map:
from arcgis.gis import GIS
gis = GIS()
map = gis.map()
df = pd.DataFrame.from_records(items)
df.columns = ['y', 'x', 'name', 'zeit']
ships = gis.content.import_data(df)
map.add_layer(ships)
map.center = [lat, lng]
map

In [ ]:
df=pd.DataFrame({'scraped_table':position})

In [ ]:
df.to_csv('Ship_positon.csv')

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account 

key_path = "authentication_key.json" # after saving I have changed the authentication key
credentials = service_account.Credentials.from_service_account_file(key_path,
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials,
project=credentials.project_id)


#Upload Data
table = 'datasets.ships_position_df' ##Replace with your destination_name.dataset.table_name
job = client.load_table_from_dataframe(df, table)

#Query Table
query_data = 'SELECT * FROM datasets.ships_position' ##Replace with your destination_name.dataset.table_name
client.query(query_data).result().to_dataframe()
